In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: C:\Users\jueming\PycharmProjects\DLEU


In [3]:
df = pd.read_stata('data/na.dta')
df.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,conm,curcd,cogs,...,intan_dc,oibdp_dc,ppegt_dc,ppent_dc,xad_dc,xlr_dc,xrd_dc,xsga_dc,dlrsn,naics
0,001000,1961-12-31,1961.0,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09,
1,001000,1962-12-31,1962.0,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09,
2,001000,1963-12-31,1963.0,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1.065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09,
3,001000,1964-12-31,1964.0,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1.474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09,
4,001000,1965-12-31,1965.0,INDL,C,D,STD,A & E PLASTIK PAK INC,USD,1.342,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09,


In [4]:
# 1. Sort by 'gvkey' and 'fyear'
df = df.sort_values(by=['gvkey', 'fyear'])

# 2. Rename 'fyear' to 'year'
df = df.rename(columns={'fyear': 'year'})

# 3. Create 'nrobs' by counting the number of observations by 'gvkey' and 'year'
df['nrobs'] = df.groupby(['gvkey', 'year'])['gvkey'].transform('size')

# 4. Drop observations for one industry (some firms are in several industries)
df = df[~(((df['nrobs'] == 2) | (df['nrobs'] == 3)) & (df['indfmt'] == "FS"))]

# 5. Sort by 'gvkey' and 'year' again
df = df.sort_values(by=['gvkey', 'year'])

# 6. Drop duplicates where 'gvkey' and 'year' are the same in consecutive rows
df = df[~((df['gvkey'] == df['gvkey'].shift()) & (df['year'] == df['year'].shift()))]

# 7. Drop firms without industry information
df = df[df['naics'].notna() & (df['naics'] != "")]


In [5]:
# 8. Generate industry codes for the first 2, 3, and 4 digits of 'naics' and convert them to numeric
for i in range(2, 5):
    df[f'ind{i}d'] = df['naics'].str[:i].astype(int)
    df[f'nrind{i}'] = df.groupby(f'ind{i}d').ngroup()

In [6]:
# 1. Create a new variable 'newmk2' which is the product of 'prcc_f' and 'csho'
df['newmk2'] = df['prcc_f'] * df['csho']

# 2. Label the new variable (This step is just a comment in Python)
# In Python, you can't directly label variables like in Stata, but you can use comments or metadata for documentation.
# "fiscal year market value prior 1998"

# 3. Replace 'mkvalt' with 'newmk2' where 'mkvalt' is missing
# df['mkvalt'] = df['mkvalt'].combine_first(df['newmk2'])
df['mkvalt'] = df['newmk2']  # Global does not have market value

# 4. Keep only the specified variables
df = df[['gvkey', 'year', 'naics', 'sale', 'cogs', 'xsga', 'xlr', 'xrd', 'xad', 'dvt', 'ppegt', 'intan', 'emp', 'mkvalt', 'oibdp'] + [col for col in df.columns if col.startswith('ind')]]

# 5. Convert the selected variables to comparable units (multiply by 1000)
vars_to_convert = ['sale', 'cogs', 'xsga', 'xlr', 'oibdp', 'mkvalt', 'dvt', 'ppegt', 'intan']
df[vars_to_convert] = df[vars_to_convert] * 1000

In [7]:
# 1. Sort by 'year'
df = df.sort_values(by='year')

# 2. Merge with a DataFrame called 'macro_df' loaded from 'macro_vars.dta'
# Assuming 'macro_df' is another DataFrame containing the 'macro_vars' data
macro_df = pd.read_stata('data/macro_vars.dta')
df = df.merge(macro_df, on='year', how='inner', suffixes=('', '_macro'))

# 3. Keep only rows where merge indicator is 3 (i.e., only rows present in both datasets)
# In pandas, the inner join automatically keeps only those rows.
# This would already be handled by the `how='inner'` in the merge above.

# 4. Deflate values by dividing by 'USGDP' and multiplying by 100
df['sale_D'] = (df['sale'] / df['USGDP']) * 100
df['cogs_D'] = (df['cogs'] / df['USGDP']) * 100
df['xsga_D'] = (df['xsga'] / df['USGDP']) * 100
df['mkvalt_D'] = (df['mkvalt'] / df['USGDP']) * 100
df['dividend_D'] = (df['dvt'] / df['USGDP']) * 100
df['capital_D'] = (df['ppegt'] / df['USGDP']) * 100
df['intan_D'] = (df['intan'] / df['USGDP']) * 100
df['xlr_D'] = (df['xlr'] / df['USGDP']) * 100

# 5. Calculate 'kexp' as the product of 'usercost' and 'capital_D'
df['kexp'] = df['usercost'] * df['capital_D']

# 6. Calculate 'mat1' as ((sale - xlr - oibdp) / USGDP) * 100
df['mat1'] = ((df['sale'] - df['xlr'] - df['oibdp']) / df['USGDP']) * 100

# 7. Trim: Drop rows where deflated 'sale_D', 'cogs_D', or 'xsga' are negative
df = df[(df['sale_D'] >= 0) & (df['cogs_D'] >= 0) & (df['xsga_D'] >= 0)]

# 8. Calculate the sales-to-COGS ratio 's_g'
df['s_g'] = df['sale'] / df['cogs']

# 9. Keep only rows where 's_g' is greater than 0
df = df[df['s_g'] > 0]

# 10. Add a 'trim' indicator column, set to 0 for all rows
df['trim'] = 0

# 11. Keep only rows where 'year' is greater than 1949
df = df[df['year'] > 1949]


In [8]:
df.to_csv('data/na_deflated.csv', index=False)

In [9]:
df.head()

,gvkey,year,naics,sale,cogs,xsga,xlr,xrd,xad,dvt,...,xsga_D,mkvalt_D,dividend_D,capital_D,intan_D,xlr_D,kexp,mat1,s_g,trim
0,002102,1954.0,311,287400.0,215610.0,56830.0,NaN,NaN,NaN,3170.0,...,371486.460668,NaN,20721.662508,3.627925e+05,0.0,NaN,NaN,NaN,1.332962,0
2,005568,1954.0,31142,234200.0,149180.0,62310.0,NaN,NaN,NaN,3350.0,...,407308.135918,NaN,21898.286877,6.059616e+05,0.0,NaN,NaN,NaN,1.569916,0
3,008585,1954.0,311,341100.0,274530.0,52240.0,39190.0,NaN,NaN,2130.0,...,341482.539245,NaN,13923.388373,4.111648e+05,0.0,256177.272454,NaN,1.879854e+06,1.242487,0
4,010597,1954.0,336611,64200.0,62860.0,0.0,NaN,NaN,NaN,1040.0,...,0.000000,NaN,6798.274135,2.307491e+05,0.0,NaN,NaN,NaN,1.021317,0
5,007127,1954.0,452111,444400.0,402490.0,0.0,NaN,0.0,NaN,11710.0,...,0.000000,NaN,76545.952040,1.024317e+06,0.0,NaN,NaN,NaN,1.104127,0


In [10]:
# Assuming the DataFrame `df` has been processed as described earlier

# 1. Calculate percentiles from 1 to 5
for t in range(1, 6):
    df[f's_g_p_{t}'] = df.groupby('year')['s_g'].transform(lambda x: x.quantile(t / 100))

# 2. Calculate percentiles from 95 to 99
for s in range(95, 100):
    df[f's_g_p_{s}'] = df.groupby('year')['s_g'].transform(lambda x: x.quantile(s / 100))

# 3. Label variables using comments (Python doesn't have direct variable labels like Stata)
# Label cogs and emp (Adding comments in your code to explain these variables)
# Costs directly allocated to production, such as material, labor, and overhead
# Number of people employed by the company and its consolidated subsidiaries in thousands

# 4. Label deflated variables with descriptions
deflate_vars = ['sale', 'cogs', 'xsga', 'mkvalt', 'intan', 'xlr']
for var in deflate_vars:
    df[f'{var}_D'] = df[f'{var}_D']  # Placeholder for labeling (use comments)

# Deflated Dvt
# Deflated cap
# Real capital expenditure

# 5. Label industry codes and other variables
# 2 digit NAICS
# 3 digit NAICS
# 4 digit NAICS
# usercost (i-delf+deprc)
# Capital expenses
# Material cost imputed
# sale-cogs ratio

# 6. Sort by 'gvkey' and 'year'
df = df.sort_values(by=['gvkey', 'year'])

# 7. Trim data based on s_g percentiles and save as CSV
df_trim = df[(df['s_g'] > df['s_g_p_1']) & (df['s_g'] < df['s_g_p_99'])]
df_trim['trim'] = 1
df_trim = df_trim.drop(columns=[col for col in df_trim.columns if col.startswith('s_g_p') or col == 'macro' or col == 'trim'])
df_trim.to_csv('data/data_main_upd_trim_1.csv', index=False)

# 8. Higher trims from 2 to 5
for t in range(2, 6):
    df_trim_t = df[(df['s_g'] > df[f's_g_p_{t}']) & (df['s_g'] < df[f's_g_p_{100-t}'])]
    df_trim_t['trim'] = t
    df_trim_t = df_trim_t.drop(columns=[col for col in df_trim_t.columns if col.startswith('s_g_p') or col == 'macro' or col == 'trim'])
    df_trim_t.to_csv(f'data/data_main_upd_trim_{t}.csv', index=False)

# Note: The `preserve` and `restore` commands in Stata are used to save the state of the dataset 
# and restore it later. In pandas, you can achieve similar functionality by creating copies of 
# your DataFrame.


C:\Users\jueming\AppData\Local\Temp\ipykernel_10620\329063752.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim['trim'] = 1
C:\Users\jueming\AppData\Local\Temp\ipykernel_10620\329063752.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim_t['trim'] = t
C:\Users\jueming\AppData\Local\Temp\ipykernel_10620\329063752.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the